In [ ]:
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import joblib
import numpy as np
import psutil
import pandas as pd
import csv
from datetime import datetime
import tflite_runtime.interpreter as tflite  # 라즈베리파이 전용

In [ ]:
# TFLite 모델 및 스케일러 로드
interpreter = tflite.Interpreter(model_path="./model/gru_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

scaler = joblib.load('./model/scaler_gru.joblib')


In [ ]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c)
ads.gain = 2/3

voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

In [ ]:
# 센서 보정값
ZMPT_offset = 2.5539
ZMPT_scale = 997.6
ACS712_offset = 2.5087
ACS712_sensitivity = 0.0990

In [ ]:
# 파생 피처 생성 함수
def extract_features(voltage_seq, current_seq):
    voltage = np.mean(voltage_seq)
    current = np.mean(current_seq)
    voltage_diff = np.abs(voltage_seq[-1] - voltage_seq[-2])
    current_diff = np.abs(current_seq[-1] - current_seq[-2])
    voltage_ma = np.mean(voltage_seq[-5:])
    current_ma = np.mean(current_seq[-5:])
    power = voltage * current
    power_diff = np.abs(power - (voltage_seq[-2] * current_seq[-2]))
    return [voltage, current, voltage_diff, current_diff, voltage_ma, current_ma, power, power_diff]


In [ ]:
def postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=8):
    if len(anomaly_buffer) < min_consecutive:
        return False
    count = 0
    for val in reversed(anomaly_buffer):
        if val == 1:
            count += 1
        else:
            break
    return count >= min_consecutive

In [ ]:
# 테스트용 데이터 (CSV)
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current']
df = pd.read_csv('./log/raw_dataset_arc.csv', names=column_names, header=None)
df = df.dropna(subset=['voltage', 'current']).reset_index(drop=True)

csv_filename = "./log/gru_log.csv"

In [ ]:
try:
    print("🚀 GRU 기반 실시간 아크 감지 시작합니다!")

    USE_SENSOR = True
    ANOMALY_THRESHOLD = 8

    data_buffer = []
    anomaly_buffer = []
    first_anomaly_detected = False
    anomaly_start_time = None
    start_time = time.perf_counter()
    idx = -1

    while True:
        idx += 1

        # 입력: 센서 또는 CSV
        if USE_SENSOR:
            try:
                v_raw = voltage_ch.voltage
                c_raw = current_ch.voltage
                voltage = (v_raw - ZMPT_offset) * ZMPT_scale
                current = (c_raw - ACS712_offset) / ACS712_sensitivity
                prev_voltage = voltage
                prev_current = current
            except (OSError, ValueError, RuntimeError) as e:
                print(f"⚠️ 센서 오류: {e}")
                voltage = prev_voltage if 'prev_voltage' in locals() else 0
                current = prev_current if 'prev_current' in locals() else 0
        else:
            if idx >= len(df):
                break
            voltage = df['voltage'].iloc[idx]
            current = df['current'].iloc[idx]

        data_buffer.append((voltage, current))
        if len(data_buffer) > 100:
            data_buffer.pop(0)

        anomaly = 0
        if len(data_buffer) >= 15:
            # 마지막 8개 시퀀스로 시계열 입력 생성
            sequence = []
            for i in range(-8, 0):
                window = data_buffer[i:i+8]
                v_seq, c_seq = zip(*window)
                features = extract_features(v_seq, c_seq)
                sequence.append(features)
            X = np.array(sequence)
            X_scaled = scaler.transform(X)
            X_scaled = X_scaled.reshape(1, 8, 8)

            interpreter.set_tensor(input_details[0]['index'], X_scaled.astype(np.float32))
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])
            y_pred = output[0][0]
            anomaly = int(y_pred > 0.5)

            anomaly_buffer.append(anomaly)
            if len(anomaly_buffer) > ANOMALY_THRESHOLD:
                anomaly_buffer.pop(0)

            if anomaly == 1 and anomaly_start_time is None:
                anomaly_start_time = time.perf_counter()
            elif anomaly == 0:
                anomaly_start_time = None

            if postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=ANOMALY_THRESHOLD):
                print("⚡ 아크 이상 감지!", idx)
                if not first_anomaly_detected and anomaly_start_time is not None:
                    total_elapsed = time.perf_counter() - start_time
                    actual_elapsed = time.perf_counter() - anomaly_start_time
                    print(f"⏱️ 전체 소요 시간: {total_elapsed:.6f}초")
                    print(f"⏱️ 연속 이상 감지까지 소요 시간: {actual_elapsed:.6f}초")
                    first_anomaly_detected = True
                    break

        # CSV 기록
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            timestamp = datetime.now().isoformat()
            writer.writerow([timestamp, voltage, current, anomaly])

        cpu = psutil.cpu_percent(interval=0.0)
        mem = psutil.Process().memory_info().rss / 1024**2
        print(f"🧠 메모리: {mem:.2f}MB | 🧮 CPU: {cpu:.2f}% | idx: {idx}")
        print(f"Voltage: {voltage:.2f}V, Current: {current:.2f}A, Anomaly: {anomaly}")

        time.sleep(0.001163)

except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")